In [1]:
import requests
import pandas as pd
import json

In [2]:
# Make a request to GET the data from the FPL API
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)

# Convert JSON data to a python object
data = json.loads(response.text)

# Create pandas DataFrame from JSON player data
playerData = pd.DataFrame.from_dict(data['elements'])

In [3]:
mini_league_url = 'https://fantasy.premierleague.com/api/leagues-classic/17177/standings/'
res = requests.get(mini_league_url)

mini_data = json.loads(res.text)

mini_league = pd.DataFrame.from_dict(mini_data['standings']['results'])

In [4]:
mini_league.head()

,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name
0,402247,56,suyash singh,1,1,1,244,58167,Aubamacare
1,402243,60,Vibhor Mehta,2,2,2,229,86437,Moonshot
2,7545404,72,Abeer Meherish,3,4,3,217,1390661,Sexual11
3,793724,47,Madhav Bhatiya,4,3,4,202,167619,GOD XI
4,35430203,35,Hridaye Nagpal,5,5,5,154,4911947,Centurions


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
playerData.shape

(576, 67)

In [7]:
playerData.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,37605,0,0,-2,2,0,3,-0.5,0.5,0,Mesut,0.0,1,False,,None,68,37605.jpg,0.0,Özil,0.8,False,None,a,1,3,0,2801,484,39826,3906,0.0,0.0,Özil,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,570,233,571,233,570,232,571,233,NaN,,NaN,,NaN,
1,100.0,75.0,39476,0,0,-1,1,0,2,-0.5,0.4,0,Sokratis,0.0,2,False,,2020-09-11T11:00:08.600094Z,49,39476.jpg,0.0,Papastathopoulos,0.1,False,None,a,1,3,0,9934,172,16466,309,0.0,0.0,Sokratis,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,486,189,478,189,451,178,490,189,NaN,,NaN,,NaN,
2,100.0,100.0,41270,0,0,0,0,0,2,0.8,1.8,2,David,1.3,3,False,,2020-09-10T14:30:11.632386Z,55,41270.jpg,1.3,Luiz Moreira Marinho,1.0,False,None,a,1,3,4,18573,3248,41422,4617,0.2,0.7,David Luiz,181,0,0,0,4,0,0,0,0,0,0,0,27,43.2,4.0,0.0,4.7,164,64,255,81,539,208,257,86,NaN,,4.0,,NaN,
3,75.0,NaN,54694,0,0,-1,1,0,3,1.9,3.5,2,Pierre-Emerick,3.0,4,False,Ankle injury - 75% chance of playing,2020-10-08T09:00:12.305506Z,119,54694.jpg,4.0,Aubameyang,23.5,False,None,d,1,3,16,325643,21445,1802222,333732,0.3,1.3,Aubameyang,360,1,1,1,5,0,0,0,1,0,0,0,54,78.2,57.2,103.0,23.8,69,26,60,44,30,13,41,22,NaN,,6.0,,1.0,
4,100.0,100.0,58822,0,0,-2,2,0,2,-0.5,0.5,0,Cédric,0.0,5,False,,2020-09-23T09:00:14.881983Z,48,58822.jpg,0.0,Soares,0.2,False,None,a,1,3,0,1981,413,26741,1252,0.0,0.0,Cédric,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,480,187,470,187,440,175,484,187,NaN,,NaN,,NaN,


In [8]:
cleaned_player_data = playerData[['first_name', 'web_name', 'element_type', 'total_points', 'now_cost', 'event_points', 'goals_scored', 'assists', 'clean_sheets', ]]

In [9]:
cleaned_player_data = cleaned_player_data.rename({'element_type': 'position', 'web_name': 'last_name'}, axis=1)
cleaned_player_data.head()

,first_name,last_name,position,total_points,now_cost,event_points,goals_scored,assists,clean_sheets
0,Mesut,Özil,3,0,68,0,0,0,0
1,Sokratis,Sokratis,2,0,49,0,0,0,0
2,David,David Luiz,2,4,55,2,0,0,0
3,Pierre-Emerick,Aubameyang,3,16,119,2,1,1,1
4,Cédric,Cédric,2,0,48,0,0,0,0


In [10]:
cleaned_player_data['position'].mask(cleaned_player_data['position'] == 1, 'GK', inplace=True)
cleaned_player_data['position'].mask(cleaned_player_data['position'] == 2, 'DEF', inplace=True)
cleaned_player_data['position'].mask(cleaned_player_data['position'] == 3, 'MID', inplace=True)
cleaned_player_data['position'].mask(cleaned_player_data['position'] == 4, 'FWD', inplace=True)

In [11]:
cleaned_player_data['now_cost'] = cleaned_player_data['now_cost'].astype(float)

In [12]:
cleaned_player_data.head()

,first_name,last_name,position,total_points,now_cost,event_points,goals_scored,assists,clean_sheets
0,Mesut,Özil,MID,0,68.0,0,0,0,0
1,Sokratis,Sokratis,DEF,0,49.0,0,0,0,0
2,David,David Luiz,DEF,4,55.0,2,0,0,0
3,Pierre-Emerick,Aubameyang,MID,16,119.0,2,1,1,1
4,Cédric,Cédric,DEF,0,48.0,0,0,0,0


In [13]:
cleaned_player_data['now_cost'] = cleaned_player_data['now_cost'] / 10

In [14]:
cleaned_player_data.head()

,first_name,last_name,position,total_points,now_cost,event_points,goals_scored,assists,clean_sheets
0,Mesut,Özil,MID,0,6.8,0,0,0,0
1,Sokratis,Sokratis,DEF,0,4.9,0,0,0,0
2,David,David Luiz,DEF,4,5.5,2,0,0,0
3,Pierre-Emerick,Aubameyang,MID,16,11.9,2,1,1,1
4,Cédric,Cédric,DEF,0,4.8,0,0,0,0


In [15]:
cleaned_player_data.sort_values('event_points', ascending=False).head(20)

,first_name,last_name,position,total_points,now_cost,event_points,goals_scored,assists,clean_sheets
43,Jack,Grealish,MID,35,7.1,24,3,3,2
61,Ollie,Watkins,FWD,23,5.9,19,3,1,2
141,Benjamin,Chilwell,DEF,18,5.6,18,1,1,1
196,James,Rodríguez,MID,35,7.8,18,3,2,1
470,Heung-Min,Son,MID,45,9.1,18,6,1,0
409,Callum,Wilson,FWD,34,6.5,16,4,1,1
468,Harry,Kane,FWD,44,10.6,16,3,6,0
124,Jorge Luiz,Jorginho,MID,27,5.2,15,3,1,1
528,Aaron,Cresswell,DEF,24,4.9,15,0,2,2
304,Mohamed,Salah,MID,41,12.2,13,5,1,1
